In [ ]:
%load_ext autoreload
%autoreload 2

# Explore

In [ ]:
from src.postprocessing.hospitals import load_gdf_hospitals
gdf = load_gdf_hospitals()

In [ ]:
# visualize
gdf[['osmid', 'name', 'name:en', 'geometry']].explore()

# Cross with predictions

In [ ]:
from src.postprocessing.utils import read_fp_within_geo
from src.constants import PREDS_PATH, RAW_PATH
from shapely.geometry import Polygon
import geopandas as gpd

OSM_RAW_PATH = RAW_PATH / "osm"

# Read raster with predictions
run_name = "240212"
folder_preds = PREDS_PATH / run_name
fp_global_preds = folder_preds / f"{run_name}_global_ukraine_preds_padded.tif"
assert fp_global_preds.exists()

# Get dataframe with hospitals (keep only polygons)
gdf = gpd.read_file(OSM_RAW_PATH / "Ukraine_hospitals.geojson")
gdf = gdf[gdf.geometry.apply(lambda x: isinstance(x, Polygon))]
gdf.shape

In [ ]:
import multiprocessing as mp
import pandas as pd
from tqdm import tqdm
from src.postprocessing.utils import get_prediction_for_geo
from src.utils.time import timeit

def _process_hospital(args):
    id_, fp, geo = args
    return dict(osmid=id_, weighted_mean=get_prediction_for_geo(fp, geo))

@timeit
def process_all_hospitals(gdf_hospitals, fp):

    args = [(row.osmid, fp, row.geometry) for _, row in gdf_hospitals.iterrows()]
    with mp.Pool(processes=mp.cpu_count()) as pool:
        preds = pool.map(_process_hospital, args)
    return gdf_hospitals.merge(pd.DataFrame(preds), on='osmid')

In [ ]:
gdf_with_preds = process_all_hospitals(gdf, fp_global_preds)
gdf_with_preds[['geometry', 'weighted_mean', 'osmid']].head()

In [ ]:
import numpy as np
threhsolds = np.arange(0.5, 1, 0.05)
n_destroyed = []
for th in threhsolds:
    n_destroyed.append((gdf_with_preds.weighted_mean > 255*th).sum())

In [ ]:
import matplotlib.pyplot as plt

_, ax = plt.subplots(figsize=(10,6))
ax.plot(threhsolds, n_destroyed)
ax.set_xlabel("Threshold")
ax.set_ylabel("Number of destroyed hospitals")
ax.set_title("Number of destroyed hospitals vs threshold")
for i, txt in enumerate(n_destroyed):
    ax.annotate(txt, (threhsolds[i], n_destroyed[i]))
plt.show()

In [ ]:
t = 0.65
hospitals_destroyed = gdf_with_preds[gdf_with_preds.weighted_mean > 255*t]
hospitals_destroyed[['osmid', 'name', 'name:en', 'geometry', 'weighted_mean']]

In [ ]:
hospitals_destroyed_ = hospitals_destroyed.copy()
hospitals_destroyed_.geometry = hospitals_destroyed_.geometry.centroid
hospitals_destroyed_[['osmid', 'name', 'name:en', 'geometry', 'weighted_mean']].explore(style_kwds={'weight':10})